In [1]:
import h5py 
import numpy as np
from keras import optimizers, regularizers, losses
from keras.models import Model
from keras import backend as K
import keras
import sys
import pickle
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"]="2"
K.tensorflow_backend._get_available_gpus()

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
SPECIFIC_FOLDER = "origGE"
dataset = "ACT_MSBBRNA_ROSMAP_PCA"

path_to_data = "/media/big/nbbwang/AD_Project/analyses/MTL_variable_tasks/MTL_data/%s/ACT_MSBBRNA_ROSMAP_PCA.h5"%SPECIFIC_FOLDER

num_components=500
with h5py.File(path_to_data, 'r') as hf:
    X = hf["ge_transformed"][:,:num_components].astype(np.float64)      

In [3]:
savepath = "../../md-ad_public_repo_data/Modeling/final_model/%s/model_transformations/"%SPECIFIC_FOLDER

curpath = "/media/big/nbbwang/AD_Project/analyses/MTL_variable_tasks/6vars-continuous"
path_to_final_models_MTL = "/media/big/nbbwang/AD_Project/analyses/MTL_variable_tasks/6vars-continuous/final_model/%s/models/"%(SPECIFIC_FOLDER)
path_to_final_models_MLPbaselines = {'ABETA_IHC': "%s/final_model/%s/models/"%(curpath,SPECIFIC_FOLDER), 
                                   'TAU_IHC': "%s/final_model/%s/models/"%(curpath,SPECIFIC_FOLDER), 
                                     'CERAD': "%s/final_model/%s/models/"%(curpath,SPECIFIC_FOLDER), 
                                   'BRAAK': "%s/final_model/%s/models/"%(curpath,SPECIFIC_FOLDER), 
                                   'PLAQUES': "%s/final_model/%s/models/"%(curpath,SPECIFIC_FOLDER), 
                                   'TANGLES':"%s/final_model/%s/models/"%(curpath,SPECIFIC_FOLDER)}

path_to_split_data = "/media/big/nbbwang/AD_Project/analyses/MTL_variable_tasks/MTL_data/%s/ACT_MSBBRNA_ROSMAP_PCASplit/"%SPECIFIC_FOLDER

path_to_final_models_chosen = "/media/big/nbbwang/AD_Project/analyses/MTL_variable_tasks/6vars-continuous/%s/final_models_chosen/"%SPECIFIC_FOLDER

MTL_final_final_model = pickle.load(open(path_to_final_models_chosen + "MTL/final.p", "rb" ) )
baselines_final_final_model = pickle.load(open(path_to_final_models_chosen + "MLP_baselines/final.p", "rb" ) )

In [4]:
phenotypes = ["CERAD", "BRAAK", "PLAQUES", "TANGLES", "ABETA_IHC", "TAU_IHC"]

In [5]:

def ignorenans_categorical_accuracy(num_categories):
    def cat_acc(y_true,y_pred):
        return
    return 

def ordloss(num_categories):
    def ordloss_cur_params(y_true, y_pred):
        return 
    return 

def ignorenans_mse(y_true, y_pred):
    bool_finite = tf.is_finite(y_true)
    return K.mean(K.square(tf.boolean_mask(y_pred, bool_finite) - tf.boolean_mask(y_true, bool_finite)), axis=-1)

def ignorenans_scaled_mse(y_true, y_pred):
    bool_finite = tf.is_finite(y_true)
    mse =  K.mean(K.square(tf.boolean_mask(y_pred, bool_finite) - tf.boolean_mask(y_true, bool_finite)), axis=-1)
    var = K.var(tf.boolean_mask(y_true, bool_finite), axis=-1)
    return mse/var
    

In [6]:
def get_model_layers(model_file, num_layers):
    
    # note: need to define custom functions for model in order to load, but these don't actually get used
    model = keras.models.load_model(model_file, custom_objects={"ordloss_cur_params": ordloss(0), \
            "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
            "ignorenans_scaled_mse": ignorenans_scaled_mse})
    
    # define new model that cuts off the last several layers
    newmodel = Model(inputs = model.input, outputs = model.layers[num_layers-1].output)
    
    # agian, need to specify these parameters, but they aren't used since we don't retrain the model
    opt = optimizers.adam()  
    newmodel.compile(optimizer=opt, loss= "mse")
    
    return newmodel

# MLP

In [7]:
MLP_LAYER_IDXes = [2,4,6,8]

In [11]:
X_transformed_layers = {}

for phenotype in phenotypes:

    fname = baselines_final_final_model[phenotype]
    

    path_to_models = path_to_final_models_MLPbaselines[phenotype] + \
                            "MLP_baselines/%s/%s/%s/"%("ACT_MSBBRNA_ROSMAP_PCA", phenotype, fname)


    repfolders_idx = np.where(["." not in x for x in  os.listdir(path_to_models)])[0]
    repfolders = np.array(os.listdir(path_to_models))[repfolders_idx]
    
    for rep in repfolders:
        path_to_model = path_to_models + "%s/200.hdf5"%rep

        full_model = keras.models.load_model(path_to_model, custom_objects={"ordloss_cur_params": ordloss(0), \
            "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
            "ignorenans_scaled_mse": ignorenans_scaled_mse})

        for HIDDEN_LAYER in [1]:

            if not os.path.isdir("%sMLP/%s/%i/"%(savepath, phenotype, HIDDEN_LAYER)):
                os.makedirs("%sMLP/%s/%i/"%(savepath, phenotype, HIDDEN_LAYER))

            # GET TRANSFORMATIONS FOR THE MODEL UP TO THE LAYER OF INTEREST 
            mod_to_layer = get_model_layers(path_to_model, MLP_LAYER_IDXes[HIDDEN_LAYER])

            X_transformed = mod_to_layer.predict(X)
            
            print(phenotype, HIDDEN_LAYER, rep)
        
            np.savetxt("%sMLP/%s/%i/%s.txt"%(savepath, phenotype, HIDDEN_LAYER, rep),X_transformed)

# MD-AD

In [9]:
MTL_LAYER_IDXes = [1, 3,\
                  {"BRAAK": 5, "CERAD":6, "PLAQUES": 7, "TANGLES":8, "ABETA_IHC":9, "TAU_IHC":10}, 
                  {"BRAAK": 17, "CERAD":18, "PLAQUES": 19, "TANGLES":20, "ABETA_IHC":21, "TAU_IHC":22},
                  {"BRAAK": 29, "CERAD":30, "PLAQUES": 31, "TANGLES":32, "ABETA_IHC":33, "TAU_IHC":34}]

In [12]:

MTL_final_final_model

path_to_models = path_to_final_models_MTL + "MTL/%s/%s/"%(dataset, MTL_final_final_model)


repfolders_idx = np.where(["." not in x for x in  os.listdir(path_to_models)])[0]
repfolders = np.array(os.listdir(path_to_models))[repfolders_idx]


for rep in repfolders:
    path_to_model = path_to_models + "%s/200.hdf5"%rep


    for HIDDEN_LAYER in [0]:

        # GET TRANSFORMATIONS FOR THE MODEL UP TO THE LAYER OF INTEREST 
        if type(MTL_LAYER_IDXes[HIDDEN_LAYER]) == dict:

            for phen in phenotypes:

                mod_to_layer = get_model_layers(path_to_model, MTL_LAYER_IDXes[HIDDEN_LAYER][phen]+1)
                X_transformed = mod_to_layer.predict(X)
                print(HIDDEN_LAYER, phen, rep)
                if not os.path.isdir("%sMTL/%i/%s/"%(savepath, HIDDEN_LAYER, phen)):
                    os.makedirs("%sMTL/%i/%s/"%(savepath, HIDDEN_LAYER, phen))
                np.savetxt("%sMTL/%i/%s/%s.txt"%(savepath, HIDDEN_LAYER, phen, rep),X_transformed)
        else:
            mod_to_layer = get_model_layers(path_to_model, MTL_LAYER_IDXes[HIDDEN_LAYER]+1)
            X_transformed = mod_to_layer.predict(X)
            
            print(HIDDEN_LAYER, rep)
            if not os.path.isdir("%sMTL/%i/"%(savepath, HIDDEN_LAYER)):
                os.makedirs("%sMTL/%i/"%(savepath, HIDDEN_LAYER))
            np.savetxt("%sMTL/%i/%s.txt"%(savepath, HIDDEN_LAYER, rep),X_transformed)